In [5]:
!pip install pyarrow
!pip install fastparquet
!pip install huggingface-hub
!pip install datasets
!pip install codebleu
!pip install tree-sitter==0.23.1
!pip install tree-sitter-python==0.21
!pip install openai

  Using cached tree_sitter-0.22.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached tree_sitter-0.22.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (544 kB)
  Attempting uninstall: tree-sitter
    Found existing installation: tree-sitter 0.23.1
    Uninstalling tree-sitter-0.23.1:
      Successfully uninstalled tree-sitter-0.23.1
  Using cached tree_sitter-0.23.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.8 kB)
Using cached tree_sitter-0.23.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (561 kB)
  Attempting uninstall: tree-sitter
    Found existing installation: tree-sitter 0.22.3
    Uninstalling tree-sitter-0.22.3:
      Successfully uninstalled tree-sitter-0.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codebleu 0.7.0 requires tree-sitter<0.23.0,>=0.22.0, b

In [6]:
import pandas as pd

splits = {'train': 'full/train-00000-of-00001.parquet', 'test': 'full/test-00000-of-00001.parquet', 'validation': 'full/validation-00000-of-00001.parquet', 'prompt': 'full/prompt-00000-of-00001.parquet'}
mbpp_df = pd.read_parquet("hf://datasets/google-research-datasets/mbpp/" + splits["test"])
mbpp_df.head()

,task_id,text,code,test_list,test_setup_code,challenge_test_list
0,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol..."
1,12,Write a function to sort a given matrix in asc...,"def sort_matrix(M):\r\n result = sorted(M, ...","[assert sort_matrix([[1, 2, 3], [2, 4, 5], [1,...",,[]
2,13,Write a function to count the most common word...,from collections import Counter\r\ndef count_c...,"[assert count_common(['red','green','black','p...",,[]
3,14,Write a python function to find the volume of ...,"def find_Volume(l,b,h) : \r\n return ((l * ...","[assert find_Volume(10,8,6) == 240, assert fin...",,[]
4,15,Write a function to split a string at lowercas...,import re\r\ndef split_lowerstring(text):\r\n ...,"[assert split_lowerstring(""AbCd"")==['bC','d'],...",,[]


In [7]:
import httpx
from openai import OpenAI
import os 
import re
from tqdm.notebook import tqdm
tqdm.pandas()


def gpt_resp(task_prompt):
    try:
        completion = client.chat.completions.create(
            model="gpt-4.1-nano",
            #prompt=Myprompt,
            messages=[
                {'role':'system', 'content': """
                You are a Python coding assistant tasked with generating incorrect or partially correct function implementations based on given definitions, docstrings, and tests.  

                ### Requirements:  
                1. **Function Completion**:  
                   - Given a function definition (with docstring), generate only the function body.  
                   - The solution must **not** fully pass the provided tests or be directly executable (e.g., contains errors, incomplete logic, or incorrect outputs).  
                
                2. **Error Diversity**:  
                   - Each solution must use a **distinct approach** (e.g., different algorithms, libraries, or logic flaws).  
                   - Include **typical Python errors** (e.g., `NameError`, `IndexError`, logical flaws, infinite loops, or incorrect edge-case handling).  
                
                3. **Formatting**:  
                   - Minimal comments (only docstrings for documentation).  
                   - No explanatory text blocks outside the function.  
                   - Separate solutions with the exact delimiter: `|@@@|`.  
                
                ### Output Example:   
                def add(a, b):  
                    return a * b  
                |@@@|  
                def add(a, b):  
                    return a + b + 1
 
                """},
                {'role': 'user', 'content': f"""Generate 15 different python solutions to accomplish the following task: {task_prompt}."""}
            ],
            max_tokens=5000,
            temperature=0
        )
        return completion.choices[0].message.content
    except:
        #print(completion)
        return 'no solution found'

mbpp_bad_refs = pd.DataFrame(columns = mbpp_df.columns)

def mbpp_bad_refs_generate(row):
    #print(row)
    global mbpp_bad_refs
    res = gpt_resp(row['text']).split('|@@@|') #[1:]
    #res = re.split(r'\n\nSolution \d+:', res)[1:] #\n\n
    if res == 'no solution found': return 
    #print(res)
    #if len(res) < 10: return 
    res_df = pd.concat([pd.DataFrame(row).transpose().assign(generated_solution=e) for e in res])
    #print(res_df)
    #print()
    mbpp_bad_refs = pd.concat([mbpp_bad_refs, res_df], ignore_index=True)
    return 


mbpp_df.progress_apply(mbpp_bad_refs_generate, axis=1)
mbpp_bad_refs.head()
#res = gpt_resp(human_eval_df.iloc[:1]['prompt'])
#res.split('|@@@|')

  0%|          | 0/500 [00:00<?, ?it/s]

,task_id,text,code,test_list,test_setup_code,challenge_test_list,generated_solution
0,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","def remove_first_last_occurrence(s, ch):\n ..."
1,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ..."
2,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ..."
3,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ..."
4,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ..."


In [34]:
import pandas as pd
mbpp_bad_refs.to_parquet('mbpp_bad_refs.parquet')
#mbpp_bad_refs = pd.read_parquet('mbpp_bad_refs.parquet')

#a=['def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    sorted_numbers = sorted(numbers)\n    for i in range(len(sorted_numbers) - 1):\n        if abs(sorted_numbers[i] - sorted_numbers[i + 1]) < threshold:\n            return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    import itertools\n    for a, b in itertools.combinations(numbers, 2):\n        if abs(a - b) < threshold:\n            return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    from bisect import bisect_left\n    sorted_numbers = sorted(numbers)\n    for num in sorted_numbers:\n        index = bisect_left(sorted_numbers, num)\n        if index > 0 and abs(num - sorted_numbers[index - 1]) < threshold:\n            return True\n        if index < len(sorted_numbers) - 1 and abs(num - sorted_numbers[index + 1]) < threshold:\n            return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    import numpy as np\n    arr = np.array(numbers)\n    for i in range(len(arr)):\n        diffs = np.abs(arr - arr[i])\n        if np.any((diffs < threshold) & (diffs > 0)):\n            return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    from itertools import combinations\n    return any(abs(a - b) < threshold for a, b in combinations(numbers, 2))\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    def quick_check(nums):\n        for i in range(len(nums)):\n            for j in range(i + 1, len(nums)):\n                if abs(nums[i] - nums[j]) < threshold:\n                    return True\n        return False\n    return quick_check(numbers)\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n', '\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    return any(abs(a - b) < threshold for i, a in enumerate(numbers) for b in numbers[i+1:])\n']
#print(a[2]), print(a[-1]), len(a)

In [18]:
mbpp_bad_refs.test_list #.iloc[0]

1       [assert first_repeated_char("abcabc") == "a", ...
2       [assert first_repeated_char("abcabc") == "a", ...
3       [assert first_repeated_char("abcabc") == "a", ...
4       [assert first_repeated_char("abcabc") == "a", ...
5       [assert first_repeated_char("abcabc") == "a", ...
                              ...                        
5071    [assert maximum_segments(7, 5, 2, 5) == 2, ass...
5072    [assert maximum_segments(7, 5, 2, 5) == 2, ass...
5073    [assert maximum_segments(7, 5, 2, 5) == 2, ass...
5074    [assert maximum_segments(7, 5, 2, 5) == 2, ass...
5075    [assert maximum_segments(7, 5, 2, 5) == 2, ass...
Name: test_list, Length: 5013, dtype: object

In [10]:
import subprocess
import sys
import pickle
import os 
from tempfile import NamedTemporaryFile

def safe_exec(func_code, timeout=30):
    """
    Safely execute Python code in an isolated subprocess.
    
    Args:
        func_code: Python code to execute (string)
        timeout: Maximum execution time in seconds
        
    Returns:
        The value of 'result' from the executed code
        
    Raises:
        TimeoutError: If execution exceeds timeout
        RuntimeError: If code execution fails
    """
    # Create a temporary file for the execution
    with NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
        # Write the code with proper result capture
        f.write(f"""
import pickle
import sys
from traceback import format_exc

try:
    local_vars = {{}}
    exec('''{func_code}''', {{}}, local_vars)
    result = local_vars.get('result', None)
    with open('{f.name}.result', 'wb') as res_file:
        pickle.dump(('success', result), res_file)
except Exception as e:
    with open('{f.name}.result', 'wb') as res_file:
        pickle.dump(('error', str(e), format_exc()), res_file)
""")
        temp_path = f.name
    
    try:
        # Run in a subprocess with resource limits
        proc = subprocess.Popen(
            [sys.executable, temp_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        try:
            stdout, stderr = proc.communicate(timeout=timeout)
            #print(stdout, stderr)
        except subprocess.TimeoutExpired:
            proc.kill()
            stdout, stderr = proc.communicate()
            raise TimeoutError(f"Execution timed out after {timeout} seconds")
        
        # Check for results
        result_file = f"{temp_path}.result"
        if os.path.exists(result_file):
            with open(result_file, 'rb') as res_file:
                status, *data = pickle.load(res_file)
                
            if status == 'success':
                return data[0]
            else:
                error_msg, traceback = data
                raise RuntimeError(f"Execution failed: {error_msg}\n\n{traceback}")
        else:
            raise RuntimeError("No result file produced. Possible crash.\n"
                              f"STDOUT: {stdout.decode()}\nSTDERR: {stderr.decode()}")
            
    finally:
        # Clean up temporary files
        for fpath in [temp_path, f"{temp_path}.result"]:
            try:
                if os.path.exists(fpath):
                    os.remove(fpath)
            except:
                pass

In [11]:
import re

def split_asserts(input_string):
    # Split the string by lines and process each line that starts with 'assert'
    lines = input_string.split('\n')
    asserts = []
    current_assert = None
    
    for line in lines:
        line = line.strip()
        if line.startswith('assert'):
            # If we have a current assert being built, add it to the list
            if current_assert is not None:
                asserts.append(current_assert)
            # Start a new assert
            current_assert = line
        elif current_assert is not None:
            # Continuation of a multi-line assert
            current_assert += ' ' + line.strip()
    
    # Add the last assert if there is one
    if current_assert is not None:
        asserts.append(current_assert)
    
    # Clean up each assert statement
    cleaned_asserts = []
    for assert_stmt in asserts:
        # Remove any trailing commas
        if assert_stmt.endswith(','):
            assert_stmt = assert_stmt[:-1]
        # Remove extra whitespace
        assert_stmt = ' '.join(assert_stmt.split())
        cleaned_asserts.append(assert_stmt)
    
    return cleaned_asserts

from concurrent.futures import ThreadPoolExecutor, TimeoutError

def count_passed_assertions(row):
    #print(row)
    function_str, prompt, canonical_solution = row['test_list'], row['text'], row['generated_solution'] #, row['entry_point'] 
    #check_function_match = re.search(r'def check\(candidate\):\n(.*?)(?=\n\s*$|\n\s*def|\n\s*@|\Z)', function_str, re.DOTALL)
    #if not check_function_match:
    #    return 0
    #check_body = check_function_match.group(1).strip()
    
    #assert_statements = split_asserts(check_body)
    assert_statements = function_str #[stmt.strip() for stmt in function_str.split('\n') if stmt.strip().startswith('assert ')]
    
    passed, total = 0, 0
    for stmt in assert_statements:
        try:
            modified_stmt = stmt #.replace('candidate', entry_point)
            full_code = f"""
{canonical_solution}

{modified_stmt}
"""
            
            total += 1
            #print(full_code)

            safe_exec(full_code, 10)
            #exec(full_code, {}, {})
            #print("PASSED")
            #print()
            passed += 1
        except TimeoutError as e:
            #print("TimeoutError", e)
            pass
        except RuntimeError as e:
            #print("RuntimeError", e)
            pass
        except MemoryError as e:
            #print("emoryError", e)
            pass
        except AssertionError as e:
            #print("AssertionError", e)
            pass
        except Exception as e:
            #print("Exception", e)
            pass
    
    return passed, total

#human_eval_bad_refs.to_parquet('human_eval_bad_refs.parquet')
#human_eval_bad_refs.shape
#human_eval_df['passed_tests'] = human_eval_df.apply(count_passed_assertions, axis=1)
#print(human_eval_bad_refs.iloc[10:11]['prompt'])
mbpp_bad_refs.iloc[10:11].progress_apply(count_passed_assertions, axis=1)
#human_eval_df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

10    (0, 3)
dtype: object

In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

mbpp_bad_refs['passed_tests'] = mbpp_bad_refs.progress_apply(count_passed_assertions, axis=1)
mbpp_bad_refs[['passed_tests', 'total_tests']] = pd.DataFrame(mbpp_bad_refs['passed_tests'].tolist(), index=mbpp_bad_refs.index)
#human_eval_bad_refs = human_eval_bad_refs[human_eval_bad_refs['passed_tests'] == human_eval_bad_refs['total_tests']]
mbpp_bad_refs.head()
'''
full_code = f"""
{human_eval_bad_refs.iloc[5]['prompt']}
{human_eval_bad_refs.iloc[5]['generated_solution']}
{human_eval_bad_refs.iloc[5]['test']}
check({human_eval_bad_refs.iloc[5]['entry_point']})
"""
#{modified_stmt}
#print(full_code)
#exec(full_code, {}, {}), 
human_eval_bad_refs.shape, human_eval_bad_refs.task_id.value_counts()
'''

  0%|          | 0/6752 [00:00<?, ?it/s]

'\nfull_code = f"""\n{human_eval_bad_refs.iloc[5][\'prompt\']}\n{human_eval_bad_refs.iloc[5][\'generated_solution\']}\n{human_eval_bad_refs.iloc[5][\'test\']}\ncheck({human_eval_bad_refs.iloc[5][\'entry_point\']})\n"""\n#{modified_stmt}\n#print(full_code)\n#exec(full_code, {}, {}), \nhuman_eval_bad_refs.shape, human_eval_bad_refs.task_id.value_counts()\n'

In [13]:
mbpp_bad_refs[mbpp_bad_refs.generated_solution == 'no solution found'].shape

AttributeError: 'DataFrame' object has no attribute 'generated_code'

In [ ]:
mbpp_bad_refs = mbpp_bad_refs[mbpp_bad_refs.generated_solution != 'no solution found']
mbpp_bad_refs.to_parquet('test_mbpp_bad_refs.parquet')

In [33]:
#mbpp_bad_refs[['passed_tests', 'total_tests']].value_counts()
mbpp_bad_refs.test_list.apply(len).value_counts()

test_list
3    5013
Name: count, dtype: int64

In [4]:
human_eval_df[['passed_tests', 'total_tests']] = pd.DataFrame(human_eval_df['passed_tests'].tolist(), index=human_eval_df.index)
human_eval_df = human_eval_df[human_eval_df['passed_tests'] == human_eval_df['total_tests']]
human_eval_df.head()

,task_id,prompt,canonical_solution,test,entry_point,passed_tests,total_tests
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements,7,7
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups,4,4
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,3,3
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero,6,6
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation,3,3


In [8]:
from codebleu import calc_codebleu
import warnings
warnings.filterwarnings("ignore")

def get_code_bleu(row):
    refined, canonical = row['generated_solution'], row['code']
    return calc_codebleu([canonical], [refined], lang="python", weights=(0.25, 0.25, 0.25, 0.25),
                         tokenizer=None)['codebleu']

    
mbpp_bad_refs['codeblue'] = mbpp_bad_refs.apply(get_code_bleu, axis=1)
mbpp_bad_refs.head()

,task_id,text,code,test_list,test_setup_code,challenge_test_list,generated_solution,codeblue
0,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","def remove_first_last_occurrence(s, ch):\n ...",0.157460
1,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ...",0.087573
2,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ...",0.115172
3,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ...",0.157835
4,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol...","\ndef remove_first_last_occurrence(s, ch):\n ...",0.087683


In [ ]:
mbpp_bad_refs[mbpp_bad_refs.generated_solution != 'no solution found'].shape

In [12]:
mbpp_bad_refs['codeblue'].value_counts().sort_index()

codeblue
0.008133    1
0.009696    1
0.013191    2
0.014728    1
0.014785    1
           ..
0.697602    7
0.705878    1
0.735014    1
0.763144    1
1.000000    1
Name: count, Length: 3410, dtype: int64

In [19]:
human_eval_bad_refs['codeblue'].value_counts().sort_index()

codeblue
0.023409    1
0.024503    1
0.026873    1
0.027262    1
0.030822    1
           ..
0.904254    2
0.927386    1
0.956659    1
0.967986    1
0.972520    1
Name: count, Length: 1549, dtype: int64

In [9]:
mbpp_bad_refs.to_parquet('test_mbpp_bad_refs.parquet')
#human_eval_bad_refs = pd.read_parquet('test_mbpp_bad_refs.parquet')

In [12]:
human_eval_bad_refs = human_eval_bad_refs[human_eval_bad_refs.generated_solution != 'no solution found']

In [40]:
human_eval_bad_refs.passed_tests.value_counts(), human_eval_bad_refs.shape, human_eval_bad_refs[human_eval_bad_refs.passed_tests != human_eval_bad_refs.total_tests].shape

(passed_tests
 0     500
 1     225
 5     219
 3     216
 2     189
 4     148
 6     121
 8     119
 7     118
 10     76
 9      52
 12     26
 16     11
 13      9
 11      4
 15      2
 Name: count, dtype: int64,
 (2035, 9),
 (1375, 9))

In [23]:
def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """

def truncate_number(number: float) -> float:
    return number - int(number)


assert truncate_number(3.5) == 0.5

In [41]:
human_eval_bad_refs[human_eval_bad_refs.passed_tests == human_eval_bad_refs.total_tests].shape

(660, 9)

In [ ]:
human_eval_bad_refs.to_parquet('human_eval_bad_refs_')